In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor

RANDOM_SEED = 42

color = sns.color_palette()
pd.options.mode.chained_assignment = None  # default='warn'

In [31]:
df = pd.read_csv("../Data/1000_samples/1000_after_pre_process.csv") 

In [10]:
api_key_path = "../api-key_geocoding.txt"
file1 = open(example1, "r")
API_KEY = file1.read()

In [11]:
import json
import urllib.request
GEOCODE_BASE_URL = "https://maps.googleapis.com/maps/api/geocode/json"
def geocode(address):
    # Join the parts of the URL together into one string.
    params = urllib.parse.urlencode({"address": address, "key": API_KEY,})
    url = f"{GEOCODE_BASE_URL}?{params}"

    result = json.load(urllib.request.urlopen(url,timeout=60))

    if result["status"] in ["OK", "ZERO_RESULTS"]:
        return result["results"]

    raise Exception(result["error_message"])


In [12]:
df['lat'] = 0.0
df['lng'] = 0.0


In [15]:
for index,row in df.iterrows():
    if index % 10 == 0:
        print(index)
    final_address = row['FINAL_ADDRESS']
    try:
        results = geocode(address=final_address)
        df.at[index, 'lat'] = results[0]["geometry"]["location"]['lat']
        df.at[index, 'lng'] = results[0]["geometry"]["location"]['lng']
    except:
        df.at[index, 'lat'] = 0.0
        df.at[index, 'lng'] = 0.0

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990


In [18]:
df.head(20)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AS OF FINAL ROLL 18/19,BLOCK,LOT,BUILDING CLASS AS OF FINAL ROLL 18/19,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,...,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,FINAL_ADDRESS,lat,lng
0,2,28,15,8,4615,23,73,10469.0,0.0,1.0,...,27500.0,4615.0,1970.0,4,72,10,2019-06-04,3260 BOSTON ROAD NYC,40.872461,-73.854265
1,3,10,0,0,6106,6,1,11209.0,1.0,0.0,...,4000.0,2816.0,1925.0,1,1,1555000,2019-10-15,9309 RIDGE BOULEVARD NYC,40.618928,-74.035543
2,3,100,10,4,2530,1156,109,11222.0,1.0,0.0,...,0.0,0.0,2017.0,2,109,2141634,2019-10-16,"21 INDIA STREET, 33B NYC",40.731916,-73.960681
3,4,12,0,0,6245,33,1,11361.0,1.0,0.0,...,3325.0,1608.0,1930.0,1,1,820000,2019-11-07,38-35 215TH PLACE NYC,40.766221,-73.767893
4,4,92,1,0,3893,31,11,11375.0,2.0,0.0,...,2367.0,1246.0,1930.0,1,11,765000,2019-10-28,89-03 70 AVENUE NYC,40.709249,-73.854456
5,3,243,1,0,878,8,9,11218.0,2.0,0.0,...,1600.0,1716.0,1905.0,1,9,2175000,2019-09-12,574 17TH STREET NYC,40.656557,-73.979849
6,3,201,12,7,7397,1105,106,11229.0,1.0,0.0,...,0.0,0.0,1976.0,2,106,324480,2019-08-20,"2362 EAST 13TH ST, 2B NYC",40.592531,-73.957405
7,3,76,6,4,5086,32,14,11226.0,16.0,0.0,...,5620.0,14000.0,1914.0,2,14,5350000,2019-06-06,"50 LINDEN BOULEVARD, 16 NYC",40.651979,-73.957153
8,5,2,4,0,6249,650,10,10312.0,2.0,0.0,...,4736.0,0.0,2019.0,1,131,986684,2019-09-26,17 HODA PLACE NYC,40.537346,-74.180734
9,4,189,1,0,3623,16,10,11385.0,2.0,0.0,...,2742.0,1760.0,1910.0,1,10,650000,2019-02-11,67-01 64TH PLACE NYC,40.706640,-73.893607


In [17]:
df.describe()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AS OF FINAL ROLL 18/19,BLOCK,LOT,BUILDING CLASS AS OF FINAL ROLL 18/19,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,lat,lng
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.0000,1000.00000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1.000000e+03,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000
mean,3.177000,123.998000,5.161000,1.944000,4711.6470,417.02600,43.93300,10888.659000,1.801000,0.428000,2.229000,8.154800e+03,3.069440e+03,1956.446000,1.503000,44.236000,1.420521e+06,39.967920,-72.603256
std,1.238226,74.425366,7.588984,2.689628,3770.9045,626.25992,49.17551,537.462155,3.476994,8.202078,8.841648,1.776215e+05,3.721923e+04,39.332075,0.815264,49.361309,2.909953e+06,5.414521,9.835172
min,1.000000,1.000000,0.000000,0.000000,16.0000,1.00000,0.00000,10001.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,1851.000000,1.000000,0.000000,1.000000e+00,0.000000,-74.252006
25%,2.000000,59.250000,0.000000,0.000000,1531.0000,25.00000,5.00000,10312.000000,1.000000,0.000000,1.000000,0.000000e+00,0.000000e+00,1925.000000,1.000000,5.000000,5.245000e+05,40.632008,-73.987877
50%,3.000000,128.500000,1.000000,0.000000,3984.0000,60.00000,11.00000,11211.000000,1.000000,0.000000,1.000000,2.000000e+03,1.397000e+03,1950.000000,1.000000,11.000000,7.499995e+05,40.696218,-73.938053
75%,4.000000,194.000000,10.000000,4.000000,6783.0000,1021.00000,109.00000,11357.000000,2.000000,0.000000,2.000000,3.000000e+03,2.217000e+03,1999.000000,2.000000,109.000000,1.226228e+06,40.753405,-73.848374
max,5.000000,247.000000,37.000000,8.000000,16283.0000,8132.00000,142.00000,11694.000000,56.000000,252.000000,252.000000,5.612000e+06,1.172005e+06,2019.000000,4.000000,142.000000,3.300000e+07,40.911264,0.000000


In [22]:
df = df[df["lat"] != 0]
df = df.drop(labels=['FINAL_ADDRESS'], axis=1)

(982, 21)

In [26]:
df.to_csv('1000_with_lat_lng.csv', index=False)